# XGB Classifier with Filter-Based FS

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [12]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/binary_new_Bacud_unbalanced_lexical.csv")      # Loading the dataset

dataset.head()

c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,url_type,url_length,url_ip_in_domain,url_domain_entropy,url_is_digits_in_domain,url_query_length,url_number_of_parameters,url_number_of_digits,url_string_entropy,url_is_https,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,16,0,3.169925,0,0,0,0,3.375000,0,...,0,0,0,0,0,0,0,0,0,0
1,0,35,0,2.807355,1,0,0,1,4.079143,0,...,0,0,0,0,0,0,0,0,0,0
2,0,31,0,2.921928,0,0,0,1,3.708093,0,...,0,0,0,0,0,0,0,0,0,0
3,1,88,0,2.896292,0,49,4,7,4.660343,0,...,0,0,1,0,0,0,0,0,0,27
4,1,235,0,3.405822,0,194,3,22,4.980518,0,...,0,0,1,0,0,0,0,0,0,27


In [13]:
dataset.head()

,url_type,url_length,url_ip_in_domain,url_domain_entropy,url_is_digits_in_domain,url_query_length,url_number_of_parameters,url_number_of_digits,url_string_entropy,url_is_https,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,16,0,3.169925,0,0,0,0,3.375000,0,...,0,0,0,0,0,0,0,0,0,0
1,0,35,0,2.807355,1,0,0,1,4.079143,0,...,0,0,0,0,0,0,0,0,0,0
2,0,31,0,2.921928,0,0,0,1,3.708093,0,...,0,0,0,0,0,0,0,0,0,0
3,1,88,0,2.896292,0,49,4,7,4.660343,0,...,0,0,1,0,0,0,0,0,0,27
4,1,235,0,3.405822,0,194,3,22,4.980518,0,...,0,0,1,0,0,0,0,0,0,27


In [14]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

#### 2. Preprocessing (Balancing)

In [15]:
dataset['url_type'].value_counts()

url_type
0    724778
1    380244
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [16]:
important_features_hybrid = ['url_is_digits_in_domain',
                             'url_query_length',
                             'url_number_of_parameters',
                             'url_string_entropy',
                             'url_path_length',
                             'url_host_length',
                             'get_tld',
                             'url_num_periods',
                             'has_exe_in_string',
                             'has_php_in_string',
                             'url_scheme']

print(len(important_features_hybrid))

X_test_lexical = x_test[important_features_hybrid]
X_train_lexical = x_train[important_features_hybrid]

11


#### Hyper-parameter Tuning

In [17]:
# Define the objective function for Optuna
def objective_lexical(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_lexical, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_lexical = optuna.create_study(direction='minimize')
study_lexical.optimize(objective_lexical, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_lexical = study_lexical.best_params
best_error_lexical = study_lexical.best_value
print("Best Hyperparameters (33 Features): ", best_params_lexical)
print("Best Error (33 Features): ", best_error_lexical)

[I 2024-04-08 12:48:29,829] A new study created in memory with name: no-name-50b0811c-d536-4724-9866-9fbc1e709507


[0]	validation-error:0.65653


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:48:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[1]	validation-error:0.39624
[2]	validation-error:0.18124
[3]	validation-error:0.11478
[4]	validation-error:0.09743
[5]	validation-error:0.09746
[6]	validation-error:0.09569
[7]	validation-error:0.09214
[8]	validation-error:0.09067
[9]	validation-error:0.08789
[10]	validation-error:0.08713
[11]	validation-error:0.08700
[12]	validation-error:0.08577
[13]	validation-error:0.08500
[14]	validation-error:0.08477
[15]	validation-error:0.08452
[16]	validation-error:0.08423
[17]	validation-error:0.08403
[18]	validation-error:0.08382
[19]	validation-error:0.08355
[20]	validation-error:0.08323
[21]	validation-error:0.08311
[22]	validation-error:0.08263
[23]	validation-error:0.08192
[24]	validation-error:0.08188
[25]	validation-error:0.08182
[26]	validation-error:0.08171
[27]	validation-error:0.08159
[28]	validation-error:0.08130
[29]	validation-error:0.08132
[30]	validation-error:0.08111
[31]	validation-error:0.08123
[32]	validation-error:0.08121
[33]	validation-error:0.08093
[34]	validation-err

[I 2024-04-08 12:48:53,978] Trial 0 finished with value: 0.27358956067315116 and parameters: {'eta': 0.2720624526840041, 'max_depth': 5, 'subsample': 0.8146910789806376, 'colsample_bytree': 0.8472236145430684, 'gamma': 7.3310254286969645, 'min_child_weight': 0.3448281386560015, 'lambda': 0.26597051366726254, 'alpha': 5.985054533852917}. Best is trial 0 with value: 0.27358956067315116.


...

[0]	validation-error:0.65653


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:57:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-04-08 12:57:16,467] Trial 98 pruned. Trial was pruned at iteration 1.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:57:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65653
[1]	validation-error:0.27658
[2]	validation-error:0.12718


[I 2024-04-08 12:57:17,123] Trial 99 pruned. Trial was pruned at iteration 2.


Best Hyperparameters (33 Features):  {'eta': 0.2923757639924323, 'max_depth': 10, 'subsample': 0.8070226122927233, 'colsample_bytree': 0.5696186734815429, 'gamma': 3.4549132350669423, 'min_child_weight': 1.4814618119792746, 'lambda': 4.3900427543904526, 'alpha': 4.466080762836852}
Best Error (33 Features):  0.2694231902709173


#### Model Training

In [18]:
from sklearn.model_selection import KFold
from xgboost import DMatrix, train

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

best_params_lexical['objective'] = 'binary:hinge'
best_params_lexical['eval_metric'] = 'error'

# Convert the data into DMatrix format
lexical_train = DMatrix(X_train_lexical, label=y_train)
lexical_valid = DMatrix(X_test_lexical, label=y_test)

# Train the Model
xgb_classifier_lexical = train(best_params_lexical, lexical_train, num_boost_round=3000)
y_pred_lexical = xgb_classifier_lexical.predict(lexical_valid)

print("Model training done.")

Model training done.


#### Evaluation

In [20]:
# Classification Report
print(classification_report(y_test, y_pred_lexical))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95    144867
           1       0.92      0.87      0.89     76138

    accuracy                           0.93    221005
   macro avg       0.93      0.91      0.92    221005
weighted avg       0.93      0.93      0.93    221005



In [22]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred_lexical, labels=xgb_classifier_lexical.classes)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_lexical.classes)
disp.plot()
plt.show()'''

AttributeError: 'Booster' object has no attribute 'classes'

In [ ]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''

"# Cross Validation Score\nscores = cross_val_score(XGBClassifier(**params_gbm),\n                        X_train, y_train, scoring='accuracy', cv=cv).mean()\n\nprint(scores)"

In [23]:
# Dumping the model
joblib.dump(xgb_classifier_lexical, 'xgb_hybrid_lexical.sav')

['xgb_hybrid_lexical.sav']

In [25]:
import lexical_generator_hybrid
import time

def xgb_predict_maliciousness(url):

    numerical_values = lexical_generator_hybrid.lexical_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_lexical.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "www.facebook.com/"
print("Current URL: "+url)

print("------------- Filter-Based (Lexical) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

Current URL: www.facebook.com/
------------- Filter-Based (Lexical) -------------
Trial 0
Malware
0.01717310003004968
Trial 1
Malware
0.02909849997377023
Trial 2
Malware
0.022313499997835606
Trial 3
Malware
0.021470999985467643
Trial 4
Malware
0.020548800006508827
Trial 5
Malware
0.019322500040289015
Trial 6
Malware
0.021581600012723356
Trial 7
Malware
0.019691599998623133
Trial 8
Malware
0.01976180000929162
Trial 9
Malware
0.020910499966703355
Trial 10
Malware
0.021918100013863295
Trial 11
Malware
0.023274099978152663
Trial 12
Malware
0.021161200013011694
Trial 13
Malware
0.022343699994962662
Trial 14
Malware
0.023142099962569773
